In [1]:
import pandas as pd
import numpy as np

## Backtesting MOVE contracts

The following analysis looks to find the optimal stop loss placements that result in the greatest overall return, as well as the frequency of what day, or what hour during the day, various stop loss levels (10%, 20%, 
30%) are most often hit. Every contract (eg. BTC-MOVE-20190925), which behaves as an option straddle, is assumed to be sold short at the beginning of each day. If a stop loss isn't hit, the contract will expire at the end of the day, and the profit will equal 'daily_open - close'. ( The backtest will also test various historical periods, ranging from 2 weeks to the whole year. Implied volatility will then be overlayed, as it is suspected that the magnitude of losses is smaller when IV is at higher levels, in hope of finding a condition that signals the best time to enter the trade.



In [2]:
df = pd.read_csv('Move_contracts_ftx_1hr.csv')
#df.set_index('Unnamed: 0', inplace = True)
df.head()

,Unnamed: 0,close,high,low,open,startTime,volume,open_btc,close_btc,contract,day,hour
0,0,106.375,106.375,104.875,105.625,2019-09-24 00:00:00+00:00,0.0,9696.00,9739.75,BTC-MOVE-20190925,24,0
1,1,107.250,110.125,106.250,106.375,2019-09-24 01:00:00+00:00,0.0,9739.75,9721.25,BTC-MOVE-20190925,24,1
2,2,106.375,107.625,104.875,107.250,2019-09-24 02:00:00+00:00,0.0,9722.00,9718.25,BTC-MOVE-20190925,24,2
3,3,105.875,106.875,104.875,106.375,2019-09-24 03:00:00+00:00,0.0,9718.25,9716.00,BTC-MOVE-20190925,24,3
4,4,104.625,105.875,104.250,105.875,2019-09-24 04:00:00+00:00,0.0,9716.00,9717.25,BTC-MOVE-20190925,24,4


In [3]:
# remove the contracts that aren't during the expiration period
mask = (df['Unnamed: 0'] >= 24) & (df['Unnamed: 0'] <= 47)
df = df.loc[mask]

In [4]:
# Calc implied volatility using black scholes, as they do on FTX
for i in df:
    df['i_vol'] = round(0.5*df['open']/(0.3989422806 * np.sqrt(1/365)*df['open_btc'])*100)

#get daily  open
for i in df:
    df['daily_open'] = df[df['Unnamed: 0']==24]['open']

#Fill the daily price forward
df['daily_open'].fillna(method='ffill', inplace= True)    
df.head()

,Unnamed: 0,close,high,low,open,startTime,volume,open_btc,close_btc,contract,day,hour,i_vol,daily_open
24,24,266.00,273.75,258.50,259.25,2019-09-25 00:00:00+00:00,2852.203550,8522.75,8705.00,BTC-MOVE-20190925,25,0,73.0,259.25
25,25,259.75,273.50,257.25,266.00,2019-09-25 01:00:00+00:00,2523.779200,8699.00,8711.25,BTC-MOVE-20190925,25,1,73.0,259.25
26,26,248.50,260.25,247.50,259.75,2019-09-25 02:00:00+00:00,121.257475,8711.75,8606.00,BTC-MOVE-20190925,25,2,71.0,259.25
27,27,243.25,251.75,242.00,248.50,2019-09-25 03:00:00+00:00,241.443400,8606.00,8590.75,BTC-MOVE-20190925,25,3,69.0,259.25
28,28,257.25,289.75,241.00,243.00,2019-09-25 04:00:00+00:00,579.500000,8590.75,8522.00,BTC-MOVE-20190925,25,4,68.0,259.25


In [5]:
# add max profit and max loss level
for i in df:
    df['max_profit'] = ((df['daily_open'] - df['low'])/df['daily_open'])*100
    
for i in df:
    df['max_loss'] = ((df['high'] - df['daily_open'])/df['daily_open'])*100

    
for i in df:
    df['PnL'] = df['daily_open'] - df['close']
    
    
df.head() 
#Unnamed: 47 is the daily profit if position not closed

,Unnamed: 0,close,high,low,open,startTime,volume,open_btc,close_btc,contract,day,hour,i_vol,daily_open,max_profit,max_loss,PnL
24,24,266.00,273.75,258.50,259.25,2019-09-25 00:00:00+00:00,2852.203550,8522.75,8705.00,BTC-MOVE-20190925,25,0,73.0,259.25,0.289296,5.593057,-6.75
25,25,259.75,273.50,257.25,266.00,2019-09-25 01:00:00+00:00,2523.779200,8699.00,8711.25,BTC-MOVE-20190925,25,1,73.0,259.25,0.771456,5.496625,-0.50
26,26,248.50,260.25,247.50,259.75,2019-09-25 02:00:00+00:00,121.257475,8711.75,8606.00,BTC-MOVE-20190925,25,2,71.0,259.25,4.532305,0.385728,10.75
27,27,243.25,251.75,242.00,248.50,2019-09-25 03:00:00+00:00,241.443400,8606.00,8590.75,BTC-MOVE-20190925,25,3,69.0,259.25,6.653809,-2.892960,16.00
28,28,257.25,289.75,241.00,243.00,2019-09-25 04:00:00+00:00,579.500000,8590.75,8522.00,BTC-MOVE-20190925,25,4,68.0,259.25,7.039537,11.764706,2.00


In [7]:
def f(row):
    for i in df['max_loss']:
        if row['max_loss'] >= 10.0:
            break
        elif row['max_loss'] < 10.0:
                val = row['PnL']
        
        return val

#need to break the loop, but continue at the start of the next day (or where df['Unnamed: 0'] = 24)
#the loop continues after the break?         
df['pnl_10'] = df.apply(f,axis = 1)
df



,Unnamed: 0,close,high,low,open,startTime,volume,open_btc,close_btc,contract,day,hour,i_vol,daily_open,max_profit,max_loss,PnL,pnl_10
24,24,266.00,273.75,258.50,259.25,2019-09-25 00:00:00+00:00,2852.203550,8522.75,8705.00,BTC-MOVE-20190925,25,0,73.0,259.25,0.289296,5.593057,-6.75,-6.75
25,25,259.75,273.50,257.25,266.00,2019-09-25 01:00:00+00:00,2523.779200,8699.00,8711.25,BTC-MOVE-20190925,25,1,73.0,259.25,0.771456,5.496625,-0.50,-0.50
26,26,248.50,260.25,247.50,259.75,2019-09-25 02:00:00+00:00,121.257475,8711.75,8606.00,BTC-MOVE-20190925,25,2,71.0,259.25,4.532305,0.385728,10.75,10.75
27,27,243.25,251.75,242.00,248.50,2019-09-25 03:00:00+00:00,241.443400,8606.00,8590.75,BTC-MOVE-20190925,25,3,69.0,259.25,6.653809,-2.892960,16.00,16.00
28,28,257.25,289.75,241.00,243.00,2019-09-25 04:00:00+00:00,579.500000,8590.75,8522.00,BTC-MOVE-20190925,25,4,68.0,259.25,7.039537,11.764706,2.00,NaN
29,29,254.25,289.00,247.25,258.25,2019-09-25 05:00:00+00:00,769.407750,8520.25,8508.75,BTC-MOVE-20190925,25,5,73.0,259.25,4.628737,11.475410,5.00,NaN
30,30,265.25,265.25,241.00,254.25,2019-09-25 06:00:00+00:00,0.000000,8508.75,8465.75,BTC-MOVE-20190925,25,6,72.0,259.25,7.039537,2.314368,-6.00,-6.00
31,31,339.00,348.00,255.00,265.25,2019-09-25 07:00:00+00:00,2646.811750,8465.75,8346.50,BTC-MOVE-20190925,25,7,75.0,259.25,1.639344,34.233365,-79.75,NaN
32,32,399.25,428.75,301.25,337.75,2019-09-25 08:00:00+00:00,6217.988500,8347.00,8266.75,BTC-MOVE-20190925,25,8,97.0,259.25,-16.200579,65.380906,-140.00,NaN
33,33,313.50,401.75,291.25,400.25,2019-09-25 09:00:00+00:00,676.512900,8265.25,8404.25,BTC-MOVE-20190925,25,9,116.0,259.25,-12.343298,54.966249,-54.25,NaN
